In [ ]:
import Clustering

#Define paths
file_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Final_UserList.txt"
train_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Final_Train/"
dest_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Clustering_"

#Create Object
obj = Clustering.Main(file_path, train_path, dest_path)

In [ ]:
#Get User List
list_users = obj.get_user_list()

In [ ]:
#Get Training Posts
df = obj.get_train_posts(list_users, 150)
display(df.head())

In [ ]:
import Text_LSTM

#Create Object
obj_lstm = Text_LSTM.Main('/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/KRM_LSTM_New_weights-40-0.87.hdf5', 
                     '/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/google_w2vec.bin')

In [ ]:
#Get embedding for captions
embedding_captions = obj_lstm.embedd_text(df['Caption'])

In [ ]:
#Get Predictions
prediction = obj_lstm.predict_text(embedding_captions)

In [ ]:
#Combine Dataframe with Text Features
df = obj_lstm.combine_text(df, prediction, "LSTM_Feature_")
display(df.head())

In [ ]:
import Image_CNN

#Import Model and Embedding
obj_cnn = Image_CNN.Main('/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/Backup/KRM_weights-260-0.69.hdf5')

In [ ]:
#Get features for images
prediction = obj_cnn.predict_image(df['Image'])

In [ ]:
#Combine Predictions 
df = obj_cnn.combine_image(df, prediction, "CNN_Feature_")
display(df.head())

In [ ]:
#Fit Model
k_posts = 4
df_post, model_images = obj.fit_posts(df, k_posts, ['Caption','File', 'Image','User'], 9001)
display(df_post.head())

In [ ]:
#Save Clusters to Local Directory
obj.save_clusters(df_post, "Posts")

In [ ]:
#Get each user's presence in each cluster 
presence_df = obj.get_cluster_presence(df_post, k_posts)
display(presence_df.head())

In [ ]:
#Fit User Presence Model
k_users = 3
df_users, model_users = obj.fit_users(presence_df, k_users, ["User"], 9001)
display(df_users.head())

In [ ]:
#Save Model
#obj.save_model(model_images, "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/model_images.plk")
#obj.save_model(model_users, "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/model_users.plk")

In [ ]:
import Predicting

#Define target path and create test object
target_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Target/"
caption_flag = True

#Create object
obj_pred = Predicting.Main(target_path, caption_flag)

In [ ]:
#Get Target Images
df_target = obj_pred.get_target_posts()
display(df_target.head())

In [ ]:
if caption_flag:
    #Get embedding for captions
    embedding_captions = obj_lstm.embedd_text(df_target['Caption'])

    #Get Predictions
    prediction = obj_lstm.predict_text(embedding_captions)

    #Combine Dataframe with Text Features
    df_target = obj_lstm.combine_text(df_target, prediction, "LSTM_Feature_")
    display(df_target.head())

In [ ]:
#Get embedding for captions
prediction = obj_cnn.predict_image(df_target['Image'])

In [ ]:
#Combine Prediction
df_target = obj_cnn.combine_image(df_target, prediction, "CNN_Feature_")
display(df_target.head())

In [ ]:
#Load Model
model_images = obj_pred.load_model("/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/model_images.plk")
model_users = obj_pred.load_model("/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/model_users.plk")

In [ ]:
#Convert to Features
if caption_flag:
    extra_cols = ['Caption','File','Image']
else:
    extra_cols = ['File','Image']
df_target_presence = obj_pred.get_cluster_presence(df_target, extra_cols, model_images)
display(df_target_presence.head())

In [ ]:
#Get Company Distances
extra_cols = ['File','Prediction']
company_final_df = obj_pred.get_dist2comp(df_target_presence, extra_cols)
display(company_final_df.head())

In [ ]:
#Get Result
extra_cols = ['File','Prediction']
target_final_df = obj_pred.get_result(df_target_presence, extra_cols, model_users)
display(target_final_df.head())